In [ ]:
import socket
import json
from datetime import datetime

# ============================================================
# ⚙️ CONFIG
# ============================================================
UDP_IP = "127.0.0.1"   # or your LAN IP (e.g., "192.168.1.5")
UDP_PORT = 5500
BUFFER_SIZE = 2048

# ============================================================
# 📡 SETUP SOCKET
# ============================================================
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))

print(f"🛰️ Listening for UDP packets on {UDP_IP}:{UDP_PORT} ...\n")

# ============================================================
# 🔁 RECEIVE LOOP
# ============================================================
try:
    while True:
        data, addr = sock.recvfrom(BUFFER_SIZE)
        ts = datetime.now().strftime("%H:%M:%S")

        try:
            msg = json.loads(data.decode())
            print(f"[{ts}] 📦 From {addr}: {json.dumps(msg, indent=2)}")
        except json.JSONDecodeError:
            print(f"[{ts}] ⚪ Raw packet from {addr}: {data.decode(errors='ignore')}")
except KeyboardInterrupt:
    print("\n🛑 Listener stopped.")
finally:
    sock.close()


🛰️ Listening for UDP packets on 127.0.0.1:5500 ...

[10:50:23] 📦 From ('127.0.0.1', 59454): {
  "source": "system",
  "command": "hover",
  "timestamp": "10:50:23"
}
[10:50:24] 📦 From ('127.0.0.1', 59454): {
  "source": "mic",
  "command": "down",
  "timestamp": "10:50:24"
}
[10:50:25] 📦 From ('127.0.0.1', 59454): {
  "source": "mic",
  "command": "down",
  "timestamp": "10:50:25"
}
[10:50:27] 📦 From ('127.0.0.1', 59454): {
  "source": "mic",
  "command": "down",
  "timestamp": "10:50:27"
}
[10:50:28] 📦 From ('127.0.0.1', 59454): {
  "source": "system",
  "command": "hover",
  "timestamp": "10:50:28"
}
[10:50:29] 📦 From ('127.0.0.1', 59454): {
  "source": "system",
  "command": "hover",
  "timestamp": "10:50:29"
}
[10:50:31] 📦 From ('127.0.0.1', 59454): {
  "source": "camera",
  "command": "up",
  "timestamp": "10:50:31"
}
[10:50:32] 📦 From ('127.0.0.1', 59454): {
  "source": "camera",
  "command": "up",
  "timestamp": "10:50:32"
}
[10:50:33] 📦 From ('127.0.0.1', 59454): {
  "source": 

In [1]:
!pip install vpython

  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
  Using cached jupyter_server-2.17.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached argon2_cffi-25.1.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached jupyter_events-0.12.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached jupyter_server_terminals-0.5.3-py3-none-any.whl.metadata (5.6 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached prometheus_client-0.23.1-py3-none-any.whl.metadata (1.9 kB)
  Using cached Send2Trash-1.8.3-py3-none-any.whl.metadata (4.0 kB)
  Using

In [3]:
"""
=========================================================
🛸 Real-Time 3D Drone Simulator — UDP Controlled (Explicit VPython)
=========================================================
Receives JSON commands via UDP (127.0.0.1:5500) and
visualizes a drone in 3D with altitude, motion trail,
and live telemetry.

Expected packet:
  {"source": "camera", "command": "forward", "timestamp": "09:15:40"}
=========================================================
"""

import socket
import json
import time
import vpython as vp

# ======================================================
# ⚙️ CONFIG
# ======================================================
UDP_IP = "127.0.0.1"
UDP_PORT = 5500
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))
sock.setblocking(False)

print(f"🛰️ Listening for UDP packets on {UDP_IP}:{UDP_PORT} ...")

# ======================================================
# 🎨 SCENE SETUP
# ======================================================
vp.scene.title = "🛸 Real-Time Drone Simulator (UDP)"
vp.scene.width = 1000
vp.scene.height = 600
vp.scene.center = vp.vector(0, 2, 0)
vp.scene.autoscale = False
vp.scene.range = 10
vp.scene.background = vp.color.black

# Drone body
body = vp.box(pos=vp.vector(0, 2, 0), size=vp.vector(1, 0.2, 1), color=vp.color.cyan)
rotor_offset = 0.6
rotor1 = vp.sphere(pos=body.pos + vp.vector(0.6, 0.1, 0.6), radius=0.1, color=vp.color.red)
rotor2 = vp.sphere(pos=body.pos + vp.vector(-0.6, 0.1, 0.6), radius=0.1, color=vp.color.green)
rotor3 = vp.sphere(pos=body.pos + vp.vector(0.6, 0.1, -0.6), radius=0.1, color=vp.color.blue)
rotor4 = vp.sphere(pos=body.pos + vp.vector(-0.6, 0.1, -0.6), radius=0.1, color=vp.color.yellow)
trail = vp.curve(color=vp.color.white)

velocity = vp.vector(0, 0, 0)
thrust = 0.0

# ======================================================
# 🧭 CONTROL MAPPING
# ======================================================
def command_to_vector(cmd: str) -> vp.vector:
    global thrust
    if cmd == "up":
        thrust = 0.03
    elif cmd == "down":
        thrust = -0.03
    elif cmd == "forward":
        return vp.vector(0, 0, -0.05)
    elif cmd == "back":
        return vp.vector(0, 0, 0.05)
    elif cmd == "left":
        return vp.vector(-0.05, 0, 0)
    elif cmd == "right":
        return vp.vector(0.05, 0, 0)
    elif cmd in ["hover", "stop"]:
        thrust = 0
        return vp.vector(0, 0, 0)
    elif cmd == "land":
        thrust = -0.02
    return vp.vector(0, 0, 0)

# ======================================================
# 🚀 MAIN LOOP
# ======================================================
last_cmd = "hover"

while True:
    vp.rate(60)
    try:
        data, _ = sock.recvfrom(1024)
        msg = json.loads(data.decode())
        cmd = msg.get("command", "hover")
        last_cmd = cmd
        print(f"[{msg.get('timestamp','--')}] Command → {cmd}")
        dv = command_to_vector(cmd)
        velocity += dv
    except BlockingIOError:
        pass

    # Update position
    velocity.y += thrust
    body.pos += velocity
    trail.append(pos=body.pos)

    # Simple drag
    velocity *= 0.95
    thrust *= 0.95

    # Ground limit
    if body.pos.y < 0.5:
        body.pos.y = 0.5
        velocity.y = 0

    # Update rotor positions
    rotor1.pos = body.pos + vp.vector(0.6, 0.1, 0.6)
    rotor2.pos = body.pos + vp.vector(-0.6, 0.1, 0.6)
    rotor3.pos = body.pos + vp.vector(0.6, 0.1, -0.6)
    rotor4.pos = body.pos + vp.vector(-0.6, 0.1, -0.6)

    # Rotor rotation animation
    for r in [rotor1, rotor2, rotor3, rotor4]:
        r.rotate(angle=0.3, axis=vp.vector(0, 1, 0), origin=body.pos)


🛰️ Listening for UDP packets on 127.0.0.1:5500 ...


KeyboardInterrupt: 